# Class Syntax

- Notice that you need `this` to access members from other members.
- Members are **public by default** to match JavaScript where everything is public.
- Note that methods are like functions by **no function keyword**.
- For the most part, you have to declare variables in the body (unlike Python).
  - the one exception is in **parameter properties** where you can declare fields in a constructor by using an access keyword like _public_.
    - interesting note: these are implemented by adding `this.x = x` to the beginning of the c'tor, which is legal in JS but not in TS
- Java-like inheritance syntax
- everything is **virtual**
- no `let` or `const` for fields
  - they all act like let by default and can be made to act const-ish with `readonly`


In [ ]:
// Worth noting: all members must be accessed with 'this' (similar to Python but without arg for it)

(() => {
  class Animal {
    // Public fields
    public name: string;
    age: number; // Public is the default to match JavaScript.

    // Private field
    private species: string;

    // Protected field
    protected habitat: string;

    // Static field
    static count: number = 0; // reset on each instance (not like Python)

    // Static method
    static getCount(): number {
      return Animal.count;
    }

    // Constant fields
    private readonly secret = "bla";
    private readonly notSet: string;

    constructor(name: string, age: number, species: string) {
      this.name = name;
      this.age = age;
      this.species = species;
      this.habitat = "Unknown";

      Animal.count++; // Increment count on each instance creation

      this.notSet = "set now"; // Can only set one time
    }

    // Getter method for species (not a property).
    public getSpecies(): string {
      return this.species;
    }

    // Setter method for habitat (not a real property).
    protected setHabitat(habitat: string): void {
      this.habitat = habitat;
    }

    introduce(): void {
      // public by default
      console.log(`Hi, I'm ${this.name} and I'm ${this.age} years old.`);
    }
  }

  class Dog extends Animal {
    // Static field
    static totalBarks: number = 0;

    // Instance property with getter
    get myBreed(): string {
      return this.breed;
    }

    // Instance property with getter and setter
    get home(): string {
      return this.habitat;
    }
    set home(home: string) {
      this.setHabitat(home);
    }

    // breed is a 'parameter property' (declared in constructor only)
    // this is often used for services and other injected things
    constructor(name: string, age: number, private breed: string) {
      super(name, age, "Dog");
      // Don't need to do anything with breed here (it's assigned already)
    }

    // Override method
    introduce(): void {
      console.log(
        `Woof! My name is ${this.name}, I'm ${this.age} years old, and I'm a ${this.breed}.`
      );
    }

    // Static method
    static getTotalBarks(): number {
      return Dog.totalBarks;
    }

    // New method
    bark(): void {
      Dog.totalBarks++;
      console.log("Woof! Woof!");
    }
  }

  const animal = new Animal("Max", 5, "Unknown");
  const dog = new Dog("Buddy", 3, "Golden Retriever");

  animal.introduce(); // Output: Hi, I'm Max and I'm 5 years old.
  dog.introduce(); // Output: Woof! My name is Buddy, I'm 3 years old, and I'm a Golden Retriever.
  dog.bark(); // Output: Woof! Woof!

  console.log(Animal.getCount()); // Output: 2
  console.log(Dog.getTotalBarks()); // Output: 1

  dog.home = "USA";
  console.log(dog.home); // Output: USA

  // Accessing private and protected properties
  //console.log(animal.species); // Error: Property 'species' is private
  //console.log(animal.habitat); // Error: Property 'habitat' is protected
})();


Hi, I'm Max and I'm 5 years old.
Woof! My name is Buddy, I'm 3 years old, and I'm a Golden Retriever.
Woof! Woof!
2
1
USA


undefined

# Nullable Members


In [4]:
(() => {
  class NullableMembers {
    optionalName?: string;
    nullableName: string | null = null;
    assertedName!: string;

    constructor() {
      // We're supposed to fill this by
      // end of constructor.
      this.assertedName = "something";
    }

    f() {
      console.log(this.optionalName);
      console.log(this.nullableName);
      console.log(this.assertedName);
    }
  }

  const n = new NullableMembers();
  n.f();
})();


undefined
null
something


undefined

# new


In [6]:
(() => {
  class MyClass {}

  const m = new MyClass(); // required
  // n = MyClass();  // Invalid
})();


undefined

# Default Constructor


In [9]:
(() => {
  class MyClass {
    // no constructor needed
  }

  const m = new MyClass();
})();


undefined

# Initializers Referencing Each Other


In [22]:
(() => {
  class MyClass {
    a = 5;
    b = this.a * this.a; // have 'this' even before c'tor
    c = this.f(); // works even though instance method
    // also note that this is not a constant expression

    f() {
      return 10;
    }
  }

  console.log(new MyClass().b);
  console.log(new MyClass().c);
})();


25
10


undefined

# super() call required!

Just like Python - the base won't be automatically called if you give a subclass constructor. However, unlike Python, the compiler will force you.

In pure JS, you'll get an error at runtime when you try to construct the derived class if you don't call the base constructor!  You cannot access `this` or return from the derived constructor without calling `super()`.

In [25]:
(() => {
  class MyBaseClass {
    constructor() {
      console.log("base called");
    }
  }

  class MyDerivedClass extends MyBaseClass {
    constructor() {
      console.log("do this first");
      super();
      console.log("derived called");
    }
  }

  new MyDerivedClass();
})();


do this first
base called
derived called


undefined

# Construction Order

It works like you'd expect from C++, mostly. Fields come before c'tor so that c'tor can refer to fields, and whole base comes before whole derived. However, one key sticking point is that instead of an initialization list, you call the base constructor in the constructor body, and that introduces some weird rules:

- you can't use `this` until after `super()` is called.
- the base fields aren't created until `super()` starts.


In [29]:
(() => {
  class MyBaseClass {
    x = this.f();
    private c = 5;

    constructor() {
      console.log(this.c);
      console.log(this.x);

      console.log("base constructor");
    }

    f() {
      console.log("base fields");
      return 0;
    }
  }

  class MyDerivedClass extends MyBaseClass {
    private c2 = 5;
    y = this.g();

    constructor() {
      // These lines won't compile because they
      // access 'this' before super().
      //console.log(this.c2);
      //console.log(this.y);

      console.log("pre-super");
      super();
      console.log("post-super");
    }

    g() {
      console.log("derived fields");
      return 0;
    }
  }

  new MyDerivedClass();
})();


pre-super
base fields
5
0
base constructor
derived fields
post-super


undefined

# Inheritance and Field Names

Fields are virtualized just like methods.

In addition, note the somewhat strange limitation that you can't define a private member with the same name in a derived class. This is probably because the emitted JS won't have the private modifier, and the names will clash.

In [41]:
(() => {
  class MyBaseClass {
    private x = 10;
    protected y = 20;
    public z = 30;
  }

  class MyDerivedClass extends MyBaseClass {
    // private x = 100;  // ILLEGAL
    protected y = 200;
    public z = 300;
  }

  const m = new MyDerivedClass();
  console.log(m.z);
  const n: MyBaseClass = m;
  console.log(n.z);
})();


300
300


undefined

# Multiple Inheritance


**Not supported** directly by the language if you're talking about actual classes. You can inherit **1 class** directly (and others indirectly in a chain) and implement **unlimited interfaces**, like C# and Java.

If you want to do a **mixin** like you'd have in C++, you have to hack it in via the constructor (eg. adding keys to the object from another object). There may be libraries for this.


# Abstract Base Class


In [48]:
(() => {
  abstract class Animal {
    // If you try to inherit Animal in a
    // non-abstract class without implementing
    // this, the compilation will fail.
    abstract makeSound(): void;

    move(): void {
      console.log("Moving...");
    }
  }

  class Dog extends Animal {
    makeSound(): void {
      console.log("Woof!");
    }
  }

  class Cat extends Animal {
    makeSound(): void {
      console.log("Meow!");
    }
  }

  // const animal = new Animal();  // ILLEGAL

  const dog = new Dog();
  dog.makeSound(); // Output: Woof!
  dog.move(); // Output: Moving...

  const cat = new Cat();
  cat.makeSound(); // Output: Meow!
  cat.move(); // Output: Moving...
})();


Woof!
Moving...
Meow!
Moving...


undefined

# Interface

Note the use of **implements** instead of **extends**.


In [51]:
(() => {
  // Define an interface
  interface Person {
    name: string; // Even fields must be 'implemented' by subclass.
    age: number;
    greet(): void; // Non-abstract implementers must implement all methods.
  }

  // Implement the interface
  class Employee implements Person {
    name: string;
    age: number;

    constructor(name: string, age: number) {
      this.name = name;
      this.age = age;
    }

    greet(): void {
      console.log(
        `Hello, my name is ${this.name} and I'm ${this.age} years old.`
      );
    }
  }

  // Create an instance of the class
  const employee: Person = new Employee("John Doe", 30);
  employee.greet(); // Output: Hello, my name is John Doe and I'm 30 years old.
})();


Hello, my name is John Doe and I'm 30 years old.


undefined

# Multiple Interfaces

TODO: find out if possible to overload in Duck where the signatures of name are different in CanFly and CanSwim (but more importantly, hope that never comes up).

Name collisions are considered same method and must be compatible.

In [58]:
(() => {
  // Define interfaces
  interface CanFly {
    fly(): void;
    // Common name in both interfaces.
    name(): void;
  }

  interface CanSwim {
    swim(): void;
    // Common name in both interfaces.
    name(): void;
  }

  // Implement interfaces
  class Bird implements CanFly {
    fly(): void {
      console.log("Bird is flying.");
    }

    // Same implementation used for both interfaces.
    name(): void {
      console.log("Bird Name.");
    }
  }

  class Fish implements CanSwim {
    swim(): void {
      console.log("Fish is swimming.");
    }

    name(): void {
      console.log("Fish Name.");
    }
  }

  class Duck implements CanFly, CanSwim {
    fly(): void {
      console.log("Duck is flying.");
    }

    swim(): void {
      console.log("Duck is swimming.");
    }

    name(): void {
      console.log("Duck Name.");
    }
  }

  // Create instances
  const bird = new Bird();
  bird.fly(); // Output: Bird is flying.

  const fish = new Fish();
  fish.swim(); // Output: Fish is swimming.

  const duck = new Duck();
  duck.fly(); // Output: Duck is flying.
  duck.swim(); // Output: Duck is swimming.
  duck.name();
})();


Bird is flying.
Fish is swimming.
Duck is flying.
Duck is swimming.
Duck Name.


undefined

# Treating Class as Interface


In [62]:
(() => {
  class MyInterface {
    greet(): void {
      console.log("Hello!");
    }
  }

  // Throw away all implementations in MyInterface
  // and treat it as a pure interface.
  class MyClass implements MyInterface {
    greet(): void {
      console.log("Greetings!");
    }
  }

  const instance: MyInterface = new MyClass();
  instance.greet(); // Output: Greetings!
})();


Greetings!


undefined

# Private Interface Members

Once you have private members on a class, you cannot use it as a pure interface. You must extend it instead.

Interfaces cannot have private things.


In [14]:
(() => {
  class Person {
    name: string;
    private age: number;
  }

  // COMPILE ERROR
  class Employee implements Person {
    name: string;
    // It doesn't matter if you include this or not.
    //private age: number;
  }
})();


Error: Line 8, Character 9
  class Employee implements Person {
________^
TS2720: Class 'Employee' incorrectly implements class 'Person'. Did you mean to extend 'Person' and inherit its members as a subclass?
  Property 'age' is missing in type 'Employee' but required in type 'Person'.

# Nested/Private Classes

You are **not allowed** to define classes inside classes or make a class private at the file level. There are tricks you can do like **barrel** them into a module and only export the ones you want to be public.


In [65]:
(() => {
  // This should ideally be inside ParentClass, but it is
  // illegal to do so, no matter what access level you say.
  class HelperClass {
    private prefix: string;

    constructor() {
      this.prefix = "Processed: ";
    }

    public process(data: number[]): string[] {
      return data.map((item) => this.prefix + item.toString());
    }
  }

  class ParentClass {
    private data: number[];

    constructor() {
      this.data = [1, 2, 3];
    }

    public processData(): void {
      const helper = new HelperClass(); // Creating an instance of the private helper class
      const processedData = helper.process(this.data);
      console.log(processedData);
    }
  }

  const parent = new ParentClass();
  parent.processData(); // Output: ['Processed: 1', 'Processed: 2', 'Processed: 3']
})();


[ 'Processed: 1', 'Processed: 2', 'Processed: 3' ]


undefined

# Structural Typing

This is where it gets really **idiomatic**. Instead of having to explicitly inherit an interface, if an object structurally matches the interface, it passes the compiler.

This is like **duck typing** but more strict.

This is a case where **instanceof** can really lead to wrong results because when an object is duck typed, it doesn't pass the manual check. To do a runtime check on this, you'd have to do your own property checks.


In [81]:
(() => {
  interface Animal {
    name: string;
    makeSound: () => void;
  }

  // Doesn't inherit Animal, but sure looks like one.
  class Dog {
    name: string;
    makeSound(): void {
      console.log("Woof!");
    }
  }

  // Only knows about Animal, the interface.
  function animalSound(animal: Animal): void {
    animal.makeSound();
  }

  // Create a dog (and thus animal) without
  // calling the constructor becuase it looks
  // right.
  const dog: Dog = {
    name: "Buddy",
    // Basically treating like an interface.
    makeSound() {
      console.log("Quack!"); // duck typed implementation
    },
  };

  animalSound(dog); // Output: "Quack!"
  animalSound({
    name: "Buddy2",
    makeSound() {
      console.log("Quack2!");
    },
  });

  //console.log(dog instanceof Animal); // Doesn't even compile!
  console.log(dog instanceof Dog); // false because of duck typing
})();


Quack!
Quack2!
false


undefined

# Structural Typing With Optional Fields

In [4]:
(() => {
  class MyClass {
      name: string;
      age?: number;
  }
  
  let x: MyClass = {name: "Fred"};
  // let y: {name: string, age: number} = x; // ILLEGAL
})();

undefined

# Structural Typing Against Classes

Classes are basically treated as interfaces in structural typing.

In [93]:
(() => {
  class MyClass {
      name: string;
      
      f() {
          console.log("MyClass");
      }
  }
  
  let x: MyClass = {name: "Fred", f: function() {console.log("x");}};
})();

undefined

# Operator Overloading

**not supported**


# Object

_All classes_ inherit from Object even if no base class.

NOTE: `.equals()` is not actually a method of `Object`, which is why primitives and collections don't have it.  It is just a convention brought over from Java where if you want to do value comparison, you can add that method.

In [5]:
(() => {
  class MyClass {
    value: number;

    constructor(value: number) {
      this.value = value;
    }

    toString(): string {
      return `MyClass { value: ${this.value} }`;
    }

    valueOf(): number {
      return this.value;
    }

    equals(other: MyClass): boolean {
      // Note that this is friendly to structural typing.
      return this.value === other.value;
    }
  }

  const obj = new MyClass(42);
  console.log(obj.toString()); // Output: MyClass { value: 42 }

  const primitiveValue = obj.valueOf();
  console.log(primitiveValue); // Output: 42

  const otherObj = new MyClass(42);
  console.log(obj.equals(otherObj)); // Output: true
  // Reference equality still fails
  console.log(obj == otherObj); // Output: false
  console.log(obj === otherObj); // Output: false
    
  const o = new Object();
})();


MyClass { value: 42 }
42
true
false
false


undefined

# Boxing

Boxing of primitive types to reference types is quite clunky compared to other languages.

It is recommended to not use the boxed types directly.  The primitives will autobox and autounbox for you.

eg. `x.toString()` boxes to call the method, but `5.toString()` is illegal because literals aren't autoboxed like that.

In [111]:
(() => {
  const x = 5;

  // This line isn't even allowed because
  // number is a primitive that doesn't
  // inherit Object and can't be treated
  // like it's in the class hierarchy.

  //console.log(x instanceof Object);

  const s: string = "hi";

  // Even string is not an Object!
  //console.log(s instanceof Object);

  // Under the hood, these convert to a
  // boxed type that inherits object!
  console.log(x.toString());
  console.log(s.toString());

  // You can use the boxed types directly
  // like this.
  const x_b = Number(x);
  const s_b = String(s);

  // But you still can't treat them like
  // instances of Object (what?????!!!!).
  //console.log(x_b instanceof Object);
  //console.log(s_b instanceof Object);
})();


5
hi


undefined

# Hashing

Unlike in other languages, TS/JS does not allow you to do value-type hashing for custom objects.  That means there is no `hashCode()` method or equivalent.  Custom objects (classes) are hashed by reference, and all instances are unique.

# Adding Members to Objects and Functions


In [6]:
(() => {
  // Python style setting on instance doesn't work.
  class MyClass {
    x = 10;
  }

  const m = new MyClass();
  //m.y = 30; // ILLEGAL

  const n = {};
  //n.y = 30; // ILLEGAL

  // Python style setting on function works.
  // Probably to support decorators.
  const f = () => {};
  f.y = 30; // LEGAL
  f();
  console.log(f.y); // Output: 30
  function fn() {}
  fn.y = 30; // LEGAL
  console.log(f.y);  // Output: 30
    
  // Dynamic object
  interface MyObject {
    [key: string]: any;

    //x: number;  // other members will become required in the {} on creation
  }
  // Even though created as a {}, using the MyObject interface
  const myObj: MyObject = {};

  myObj.field1 = "value1";
  myObj.field2 = 42;
  myObj.field3 = { foo: "bar" };

  console.log(myObj.field1); // Output: value1
  console.log(myObj.field2); // Output: 42
  console.log(myObj.field3); // Output: { foo: 'bar' }
})();


30
30
value1
42
{ foo: 'bar' }


undefined

# Object Indexer

Maps don't have the indexer syntax like in Python, but objects do!

**WARNING**: mixing of string names and syntax names for object members can cause problems with **minification**. The syntax access will get minified but the strings will not. So something that works in a dev build may not work in production.


In [95]:
(() => {
  const myObj = {
    field1: "value1",
    field2: 42,
    field3: { foo: "bar" },
  };

  const fieldName = "field1";
  console.log(myObj[fieldName]); // Output: value1

  const dynamicField = "field2";
  console.log(myObj[dynamicField]); // Output: 42
})();


value1
42


undefined

# Copying Objects


In [98]:
(() => {
  const myObj = {
    field1: "value1",
    field2: 42,
    field3: { foo: "bar" },
  };

  const myCopy = {
    ...myObj,
    field3: "replaced", // updates field copied in
    field4: "new", // brand new field
    // Note that if you did the ... here, field3 would
    // get set back to the myObj version.
  };

  myObj.field1 = "newvalue1"; // no effect on myCopy
  console.log(myCopy);
})();


{ field1: 'value1', field2: 42, field3: 'replaced', field4: 'new' }


undefined

# Ommitting Key Names

This works because {} is not used as a set like in other languages. So when the compiler sees {}, it already knows it's an object.


In [102]:
(() => {
  const field1 = 10;
  const field2 = 20;
  const field3 = 30;

  const myObj = {
    // If we already have a variable with the same
    // name, we can just say it once.
    field1,
    field2,
    // Or we can set it explicitly too.
    // If the name is different, we have to.
    field3: field3,
  };

  console.log(myObj);
})();


{ field1: 10, field2: 20, field3: 30 }


undefined

# Declaration Order

TypeScript is **pretty flexible** about order of declarations at the same level.

Also note that inner names can **hide outer** names as you'd expect in any language.

However, you **cannot redeclare the same name at the same level**, which is why all my jupyter cells have immediately called lambdas.


In [115]:
(() => {
  // We'll define f later.
  const x = f();

  function f() {
    return 5;
  }

  console.log(x);
})();


5


undefined

In [117]:
(() => {
  // Obviously this can't work.
  //const x = y;
  const y = 5;
})();


undefined

In [120]:
(() => {
  class A {
    f(): B {
      return new B();
    }
  }

  class B {}

  const a = new A();
  a.f();
})();


undefined

In [122]:
(() => {
  // This one doesn't work
  /*
  class A extends B {
  }
  
  class B {
  }
  
  const a = new A();
  */
})();


undefined

In [2]:
(() => {
  // This is even easier because when you say 'this' it already
  // knows where to look.
  class MyClass {
    x = this.f();

    g() {
      return this.f();
    }
    f() {
      return 5;
    }
  }
})();


undefined

# String Keys


When creating an object instance, you can provide strings intead of straight keys as the keys.

An important use case for this is to get around **minification**. Any symbol in code can be changed when the code is minified, so if you depended on a specific key existing by name, a minified version of the call may check the wrong key. An example of this is in **reactive forms** when you've specified the names of fields both in code and in the HTML template.

This goes together with using **ambient interfaces** (see Modules & Imports notebook) to declare interfaces that won't get minified when you need to pass them around in code.


In [6]:
(() => {
  interface MyInterface {
    x: number;
    y: number;
  }

  const m = {
    x: 10, // string as key
    y: 20, // normal key
  };
  console.log(m); // equivalent
})();


{ x: 10, y: 20 }


undefined

You can also do this on a class definition, but it's weird and probably not a good idea.


In [4]:
(() => {
  class MyClass {
    "a" = 5;
  }

  const m = new MyClass();
  console.log(m.a);
})();


5


undefined

# Partial Classes

** not supported **


# Partial<\T\>

This type is syntactic magic for turning an interface into the same interface with **all optional fields**.


In [9]:
(() => {
  interface MyInterface {
    x: number;
    y: number;
    z?: number;
  }

  // This would have an error if you didn't use Partial.
  const m: Partial<MyInterface> = {};
  m.x = 10;
  console.log(m);
})();


{ x: 10 }


undefined

# 'Override' keyword

This is a **newer** addition to TypeScript for explicitly specifying that you are **overriding a base method**.

By default, it is optional, but there is a compiler option (`noImplicitOverride`) to make it mandatory, which **Angular** turns on by default.


In [10]:
(() => {
  class MyBaseClass {
    f(): void {}
  }

  class MySubClass extends MyBaseClass {
    override f(): void {
      console.log("overridden!");
    }
  }

  const m = new MySubClass() as MyBaseClass;
  m.f();
})();


overridden!


undefined

In [16]:
(() => {
  class MyBaseClass {
    f(): void {}
  }

  class MySubClass implements MyBaseClass {
    // override keyword NOT ALLOWED if
    // interface instead of base class
    f(): void {
      console.log("overridden!");
    }
  }

  const m = new MySubClass() as MyBaseClass;
  m.f();
})();


overridden!


undefined

# Index Signatures

## JS Framework

First, it's helpful to know what index signatures are in reference to.  It is not actually functionality added to a class but is a type safety feature.

In pure JS, all reference types have an index operator `[]` that automatically lets you index the attributes on the instance by name.  `o['x']` means the same thing as `o.x`.

In pure JS, you're allowed to set those values as well as retrieve them.  You can add arbitrary attributes to an instance at runtime via either the `.` operator or `[]`, and they stay in sync.

For an array, `arr[0]` and `arr['toString']` both work using the same operator technically.

For a map, `m[key]` is __not supported__ as you'd expect (just has the object properties and not the keys).

In [8]:
// this cell won't work in TypeScript!
(() => {
    const o = new Object();
    o.x = 10;
    console.log(o.x); // 10
    console.log(o['x']); // 10
    
    o[y] = 100;
    console.log(o[y]); // 100
    console.log(o.y); // 100
    
    console.log(o[z]); // undefined
})();

Error: Line 4, Character 7
    o.x = 10;
______^
TS2339: Property 'x' does not exist on type 'Object'.

Line 5, Character 19
    console.log(o.x); // 10
__________________^
TS2339: Property 'x' does not exist on type 'Object'.

Line 8, Character 7
    o[y] = 100;
______^
TS2304: Cannot find name 'y'.

Line 9, Character 19
    console.log(o[y]); // 100
__________________^
TS2304: Cannot find name 'y'.

Line 10, Character 19
    console.log(o.y); // 100
__________________^
TS2339: Property 'y' does not exist on type 'Object'.

Line 12, Character 19
    console.log(o[z]); // undefined
__________________^
TS2304: Cannot find name 'z'.

## TS Modification

TypeScript prevents you from accessing a member with `.` that is not supposed to exist on the class.

It also seems to sometimes check the specific strings passed into `[]` to make sure they're valid, at compile-time.  For instance, in TypeScript Playground, it throws an error for `m['y'] = 100`, but in Jupyter, it does not.  There must be a compile option for that.

Retrieving a non-existent field with `[]` returns `undefined` in Jupyter but is a compile error in TS Playground.

In [17]:
(() => {
    class MyClass {x: number;}
    
    const m = new MyClass();
    m.x = 10;
    // m.y = 100; // ILLEGAL
    m['y'] = 100; // LEGAL (here, but not in TS playground)
    // console.log(m.y); // ILLEGAL
    console.log(m['y']); // 100
    console.log(m['z']); // undefined
})();

100
undefined


undefined

## Index Signature

This is strictly a __compiler feature__ to give you back some of the flexibility that JS gives objects while still maintaining control of the situation.

You can define an index signature on an interface or class, and all it means is the compiler will let you use `[]` or `.` the way JS would to either set or get a value, as long as it conforms to the types given in the index signature.  You could use `any` to make it fully flexible like JS.  You can also use `readonly` to make the TS compiler prevent writing that way.

Note that other properties included in the object also behave as part of the indexer, so if they contradict the signature, the compiler will complain.  But that only applies if the key part of the indexer is the same type as the property - otherwise it will not complain.  You can use type unions for more flexibility.

When used on a class, the indexer is still abstract-looking just like an interface and works the same way, but it means base classes and subclasses will be checked against the indexer too.  Members of `Object` are exempt from the check because they're fundamental to the language.

In [74]:
(() => {
    // simple case on interface
    interface MyInterface {
        [index: number]: number;
        
        x: number; // string to number, but accepted
    }
    let i: MyInterface = {x: 20}; // OK
    // i = {x: 20, y: 10}; // NOT OK because y is checked against indexer
    
    // conflict on interface
    interface MyInterface2 {
        [index: string]: number;
        
        x: number; // OK
        // y: string; // NOT OK because triggers left side of the indexer check
    }
    
    // type union
    interface MyInterface3 {
        [index: string|number]: string|number;
        
        x: string;
        y: number;
    }
    let j: MyInterface3 = {x: 'hi', y: 20}; // OK
    j[0] = 'bob'; // OK
    
    // dynamic values
    interface MyInterface4 {
        [index: string]: string;
        
        x: string;
    }
    let k: MyInterface4 = {x: 'a', y: 'b'}; // OK
    k.z = 'c'; // OK
    
    // readonly
    interface MyInterface5 {
        readonly [index: string]: string;
        
        x: string;
    }
    let l: MyInterface5 = {x: 'a', y: 'b'}; // OK despite readonly
    // l.z = 'c'; // NOT ALLOWED
    // l['z'] = 'c'; // NOT ALLOWED
 
    // class instead of interface
    class MyClass {
        [index: string]: string; // no "implementation", still works the same way
    }
    let m: MyClass = new MyClass();
    m.x = 'hi'; // OK
    
    // subclass of class with indexer
    class MyDerivedClass extends MyClass {
        // x: number; // not allowed because conflicts with indexer
    }
    
    // subclass adding indexer
    class MyBase {
        x: number;
    }
    class MyDerived extends MyBase {
        // [index: string]: string; // not allowed because conflicts with base
    }
    
    // conflicts with Object?
    // no - typescript automatically exempts those
    class MyObject {
        [index: string]: number;
    }
})();

undefined

# Order of Members in Structural Typing

Types are considered compatible __without considered member order__.

In [76]:
(() => {
    interface MyInterface {
        x: number;
        y: number;
    }
    
    const i: MyInterface = {y: 20, x: 10}; // OK
})();

undefined

# {} literals and Object

All objects inherit from `Object` and have the appropriate members, even if they look like just interfaces or anonymous classes.  You don't need to think too hard about this because all objects have the properties, so it doesn't interfere with structural typing.

In [77]:
(() => {
    interface MyInterface {}
    
    const i: MyInterface = {};
    console.log(i.toString());
})();

[object Object]


undefined

# Anonymous Classes

NOTE: anonymous classes are __closures__ just like lambdas are.

In [35]:
(() => {
    function f() {
        return {x: 10, y: 20, z: 30, 
                volume: function() {return this.x * this.y * this.z;}
               };
    }
    
    console.log(f());
    console.log(f().volume());
})();

{ x: 10, y: 20, z: 30, volume: [Function: volume] }
6000


undefined

# Polymorphism

In [90]:
(() => {
    class MyBaseClass {}
    class MyClass extends MyBaseClass {}
    
    function getBase(): MyBaseClass {
        return new MyClass();
    }
    
    const b: MyBaseClass = getBase();
    // const m: MyClass = b; // won't work if strict type checks enabled (jupyter has them off)
    const m: MyClass = b as MyClass; // the proper way
    console.log(b instanceof MyClass); // works if you create classes with 'new'
    
    if (b instanceof MyClass) {
        const n: MyClass = b;  // ok due to type narrowing
    }
    
    const f: MyClass = {}; // structural typing
    console.log(f instanceof MyClass); // this is FALSE!
})();

true
false


undefined

# Computed Properties

This is a __JS feature__ that works pretty much unchanged in TS. It works for any object-like thing with keys (objects, interfaces, classes).

Basically, it's an extension of the fact that all objects have the `[]` operator as an alternative way to access keys.  Instead of giving a key name, quoted or not, you put `[]` around the key part.  What's inside is some expression, which can be used again with the `[]` operator on the instance to retrieve that value.

Any normal syntax thing you could do on a key-value pair in an object or class (such as type annotations, functions, variables, etc.) will work, but you just use the whole thing in `[]` as the name.

The part in `[]` is a computed expression, so it can be dynamically computed or be something that is not a string such as a number.

When you iterate the keys of the object, it will give you the actual items used.

In [19]:
(() => {
    const t = 'Hello, ';
    
    const m = {
        ['x']: 100,
        [20]: 200,
        [30]: 300,
        [40](a: number) { return a*a; },
        [t + 'Bob']: 400,
        'a b c ': 500,
    };
    
    console.log(m); // all the above shown
    console.log();
    
    console.log(m.x);
    console.log(m['x']);
    // console.log(m.20); // NO
    console.log(m[20]);
    console.log(m[30]);
    console.log(m[40](10)); // 100 (10 squared)
    console.log(m['Hello, Bob']);
    
    console.log();
    for (const item of Object.keys(m)) {
        console.log(item);
    }
})();

{
  '20': 200,
  '30': 300,
  '40': [Function (anonymous)],
  x: 100,
  'Hello, Bob': 400,
  'a b c ': 500
}

100
100
200
300
100
400

20
30
40
x
Hello, Bob
a b c 


undefined

# Iterating Over Properties

When you iterate over the keys of an object using `Object.keys(o)`, it will only pick up keys that are actually set on the object, not variables that are declared in the class but not set.  But even setting explicitly to `undefined` counts!  Functions are not included, but lambda variables are, including function expressions!

In [31]:
(() => {
    class MyClass {
        x = 10;
        y: number = 20;
        z: number; // technically not set on instance
        
        [1] = 30;
        [2] = undefined; // set but set to undefined (not same thing)
        [3] = null;
        
        f() {}
        g = () => {}
        h = function() {}
    }
    
    class MyDerivedClass extends MyClass {
        x = 20;
        f() {}
    }
    
    const m = new MyClass();
    for (const item of Object.keys(m)) {
        console.log(item);
    }
    console.log(m.z);
    console.log();
    
    const n = new MyDerivedClass();
    for (const item of Object.keys(n)) {
        console.log(item);
    }
})();

1
2
3
x
y
g
h
undefined

1
2
3
x
y
g
h


undefined

# Iterator Protocol

This can't be shown in Jupyter because it requires ES6 symbols!  It is a pure JS feature.

To make a class iterable (eg. with `for...of`), it needs a method called `[Symbol.iterator]()` that takes no params and returns an object that has a `next()` method that returns an `IteratorResult<T>`.  `IteratorResult` is an interface defined in TypeScript (not JS) to describe the structure expected to be returned from `next()`, which is an object that has a `value` and `done` property.  Return `null` and `true` when there are no more items to return.

Here is an example that uses an anonymous class:

```
class IterableClass {
    private items: number[] = [];

    constructor(items: number[]) {
        this.items = items;
    }

    // Implementing the [Symbol.iterator] method
    [Symbol.iterator]() {
        let index = 0;
        let items = this.items;

        return {
            next(): IteratorResult<number> {
                if (index < items.length) {
                    return { value: items[index++], done: false };
                } else {
                    return { value: null, done: true };
                }
            }
        };
    }
}

// Example usage
const myIterable = new IterableClass([1, 2, 3, 4, 5]);

for (const item of myIterable) {
    console.log(item); // Outputs 1, 2, 3, 4, 5
}
```

NOTE: you can also put a `*` in front to use it as a __generator__ function too (with no anonynous object needed).

# Generator Methods

Works just like a generator function, but the `*` goes in front of the name instead of after the word `function` since you don't have that in a method.

# Summary of Object-Like Entity Syntax Differences

## Terminology

In general, __type literals__ and __interfaces__ act the same and are both TypeScript-only entities.

__Classes__ and __anonymous objects__ are from JS and behavior differently from each other and from the TypeScript-onlys stuff.

In [37]:
(() => {
    // Type Literal
    type Person = {
        name: string;
        age: number;
    };
    
    // Type Literal (again)
    let person: {
        name: string;
        age: number;
    };

    // Anonymous Object
    let person2 = {
        name: "Alice",
        age: 30
    };

    // Type Literal + Anonymous Object
    let person3: { name: string; age: number; } = { name: "Alice", age: 30 };

    // Class
    class Person2 {
        name: string;
        age: number;

        constructor(name: string, age: number) {
            this.name = name;
            this.age = age;
        }
    }

    // Interface
    interface IPerson {
        name: string;
        age: number;
    }

    // Object Destructuring (and Type Literal)
    function greet({ name, age }: { name: string; age: number }) {
        console.log(`Hello, my name is ${name} and I am ${age} years old.`);
    }

})();

undefined

## Functions

__Function expressions__ are only for assigning a function definition on the right side, as an alternative to an arrow lambda.  They have no consideration on the left side (the type).

__Method signatures__ are the name for when you define a member function like in other languages, while __function type property__ is the name for a variable that can hold a lambda or other function-like object.

In [60]:
(() => {
    // Type Literal
    type Person = {
        f(); // method signature
        g: () => void; // function type property
    };
    
    // Type Literal (again)
    let person: {
        f(); // method signature
        g: () => void; // function type property
    };

    // Anonymous Object
    let person2 = {
        // note the required COMMAS in this case, which you may not expect
        f() {}, // method
        g: function() {}, // function expression
        h: () => {} // lambda
    };

    // Type Literal + Anonymous Object
    // right side could be function expression too
    let person3: {f(); g: () => void} = { f() {}, g: () => {} };

    // Class
    class Person2 {
        f() {} // lack of comma or semicolon is unique from above
        g = function() {}
        h = () => {}
    }

    // Interface
    interface IPerson {
        f();
        g: () => void;
    }
})();

undefined

## Comma vs. Semicolon Separating Members

Pure TypeScript things (__type literal__ and __interface__) can take either one, but are typically used with semicolon.

__Classes__ can only do semicolon, but it is not needed after `{}` for function bodies, even if function expression or lambda.

__Anonymous objects__ always have to use commas, even for function bodies.

In [84]:
(() => {
    // Type Literal
    // either one but usually semicolon
    // technically could be neither due to optional ; in JS
    type Person = {
        f();
        g: () => void;
        x: number,
        y: number,
        h();
    };

    // Anonymous Object
    // comma only (and always, even for functions, never nothing)
    let person2 = {
        x: 20,
        f() {},
        g: function() {},
        h: () => {}
    };

    // Type Literal + Anonymous Object
    // right side could be function expression too
    // either fine (comma often looks better since matches right side)
    let person3: {f(), g: () => void} = { f() {}, g: () => {} };

    // Class
    // semicolon only (and not after function bodies, though you could)
    class Person2 {
        x: number;
        
        f() {};
        g = function() {}
        h = () => {}
        
        i: () => void;
    }

    // Interface
    // either one
    interface IPerson {
        f();
        g: () => void,
        x: number,
    }
})();

undefined

## = vs : and Types

Colons are usually used for types and equal is usually used for values, with these exceptions:
  - for methods colon gives return type
    - if no return type, them colon not needed for the name and params
  - since interfaces and type literals have no implementations, they won't use equals
  - __anonymous objects__ are special in that they only use equals and no types (except maybe method return type)

In [92]:
(() => {
    // Type Literal
    // semicolon used for type (if not method signature, unless return type)
    // no = because no implementations
    type Person = {
        f(): void;
        g: () => void;
        x: number,
        y: number,
        h();
    };

    // Anonymous Object
    // : only (and not needed for method signature)
    // no types (unless provided in method signatures or function expression return type)
    let person2 = {
        x: 20,
        f(): void {},
        g: function() {},
        h: () => {}
    };

    // Class
    // colon for type, = for value (except method signature)
    class Person2 {
        x: number;
        
        f() {};
        g = function() {}
        h = () => {}
        
        i: () => void;
    }

    // Interface
    // semicolon used for type (if not method signature, unless return type)
    // no = because no implementations
    interface IPerson {
        f();
        g: () => void,
        x: number,
    }
})();

undefined

## this

__Interfaces__ and __type literals__ don't interact with `this` because they don't have function bodies in them.  it is only a matter for __classes__ and __anonymous objects__.

Inside a __class__, `this` is the class instance (shadowing the global object). This is the case for all 3 ways of defining functions.  It is also the case for lambdas inside class methods, which is useful for returning those to be called elsewhere.  Note that by definition, `static` methods don't bind `this` to the class instance.  They bind to the class itself instead.

Inside an __anonymous object__, it depends on the type of function. A lambda (arrow function) takes `this` from the surrounding scope (eg. the class containing the anonymous object, or the global object otherwise).  A method or function expression sets `this` to the anonymous object itself, making it act as a true method of the anonymous object as a class instance of its own.

Functions that aren't in classes or anonymous objects just use the global object.

In [107]:
(() => {
    // Anonymous Object
    let person = {
        name: 'person',
        
        f() {
            console.log(this);
        },
        
        g: () => {
            console.log(this);
        },
        
        h: function() {
            console.log(this);
        },
    };

    // Class
    class Person {
        readonly name = 'class Person';
        
        f() {
            console.log(this);
        }
        
        g = function() {
            console.log(this);
        }
        
        h = () => {
            console.log(this);
        }
    }
    
    function f() {
        console.log(this);
    }
    
    const g = function() {
        console.log(this);
    }
    
    const h = () => {
        console.log(this);
    }
    
    person.f(); // person (method)
    person.g(); // global (lambda)
    person.h(); // person (function expression)
    
    const m = new Person();
    m.f(); // m
    m.g(); // m
    m.h(); // m
    
    f(); // global
    g(); // global
    h(); // global
})();

{
  name: 'person',
  f: [Function: f],
  g: [Function: g],
  h: [Function: h]
}
<ref *1> Object [global] {
  global: [Circular *1],
  queueMicrotask: [Function: queueMicrotask],
  clearImmediate: [Function: clearImmediate],
  setImmediate: [Function: setImmediate] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  structuredClone: [Getter/Setter],
  clearInterval: [Function: clearInterval],
  clearTimeout: [Function: clearTimeout],
  setInterval: [Function: setInterval],
  setTimeout: [Function: setTimeout] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  atob: [Getter/Setter],
  btoa: [Getter/Setter],
  performance: [Getter/Setter],
  fetch: [AsyncFunction: fetch],
  crypto: [Getter],
  __filename: '[eval]',
  module: <ref *2> [Function: Module] {
    _cache: [Object: null prototype] {},
    _pathCache: [Object: null prototype] {},
    _extensions: [Object: null prototype] {
      '.js': [Function (anonymous)],
      '.json': [Function (anonymous)],
      '.no

<ref *1> Object [global] {
  global: [Circular *1],
  queueMicrotask: [Function: queueMicrotask],
  clearImmediate: [Function: clearImmediate],
  setImmediate: [Function: setImmediate] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  structuredClone: [Getter/Setter],
  clearInterval: [Function: clearInterval],
  clearTimeout: [Function: clearTimeout],
  setInterval: [Function: setInterval],
  setTimeout: [Function: setTimeout] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  atob: [Getter/Setter],
  btoa: [Getter/Setter],
  performance: [Getter/Setter],
  fetch: [AsyncFunction: fetch],
  crypto: [Getter],
  __filename: '[eval]',
  module: <ref *2> [Function: Module] {
    _cache: [Object: null prototype] {},
    _pathCache: [Object: null prototype] {},
    _extensions: [Object: null prototype] {
      '.js': [Function (anonymous)],
      '.json': [Function (anonymous)],
      '.node': [Function (anonymous)]
    },
    globalPaths: [
      '/Users/davidpetrofsk

undefined

# 'this' callsite vs. definition site

In this example, you can see that f, g, and i always have the same result as each other in a given use case - because __functions__, __method signatures__, and __function expressions__ have the same behavior when it comes to `this`.  The dichotomy is only between those things vs. __arrow functions__.

The difference is that __arrow functions__ bind `this` based on their __definition__ whereas all the others bind it based on their __call site__.

But you have to keep in mind that __call site__ means __how it's called__, not what `this` means when it's called.  For instance, in the example below, we defined f and g at the top level.  If you assign variables to reference them inside an object, and then treat them as methods of that object, then `this` is bound to that object.  But simply calling those functions inside a function where `this` is bound to something else is meaningless (gets global object, not inheriting `this`).

A function or function express can become an instance method just by taking a variable to it in the class, or by binding it explicitly.  An arrow function doesn't have this behavior because it's meant to be passable around and continue to close around what it saw on creation.

In [8]:
(() => {
    function f() {
        console.log(this);
    }
    
    const g = function() {
        console.log(this);
    }
    
    const h = () => console.log(this);
    
    class MyClass {
        makeObject() {
            return {
                f, g, h, i: function() {console.log(this);}, j: () => console.log(this),
            };
        }
        
        testF() {
            f();
        }
    }
    
    const m = new MyClass().makeObject();
    
    f(); // global object
    g(); // global object
    h(); // global object
    
    m.f(); // anonymous object
    m.g(); // anonymous object
    m.h(); // global object
    m.i(); // anonymous object
    m.j(); // MyClass
    
    new MyClass().testF(); // global object
})();

<ref *1> Object [global] {
  global: [Circular *1],
  queueMicrotask: [Function: queueMicrotask],
  clearImmediate: [Function: clearImmediate],
  setImmediate: [Function: setImmediate] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  structuredClone: [Getter/Setter],
  clearInterval: [Function: clearInterval],
  clearTimeout: [Function: clearTimeout],
  setInterval: [Function: setInterval],
  setTimeout: [Function: setTimeout] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  atob: [Getter/Setter],
  btoa: [Getter/Setter],
  performance: [Getter/Setter],
  fetch: [AsyncFunction: fetch],
  crypto: [Getter],
  __filename: '[eval]',
  module: <ref *2> [Function: Module] {
    _cache: [Object: null prototype] {},
    _pathCache: [Object: null prototype] {},
    _extensions: [Object: null prototype] {
      '.js': [Function (anonymous)],
      '.json': [Function (anonymous)],
      '.node': [Function (anonymous)]
    },
    globalPaths: [
      '/Users/davidpetrofsk

<ref *1> Object [global] {
  global: [Circular *1],
  queueMicrotask: [Function: queueMicrotask],
  clearImmediate: [Function: clearImmediate],
  setImmediate: [Function: setImmediate] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  structuredClone: [Getter/Setter],
  clearInterval: [Function: clearInterval],
  clearTimeout: [Function: clearTimeout],
  setInterval: [Function: setInterval],
  setTimeout: [Function: setTimeout] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  atob: [Getter/Setter],
  btoa: [Getter/Setter],
  performance: [Getter/Setter],
  fetch: [AsyncFunction: fetch],
  crypto: [Getter],
  __filename: '[eval]',
  module: <ref *2> [Function: Module] {
    _cache: [Object: null prototype] {},
    _pathCache: [Object: null prototype] {},
    _extensions: [Object: null prototype] {
      '.js': [Function (anonymous)],
      '.json': [Function (anonymous)],
      '.node': [Function (anonymous)]
    },
    globalPaths: [
      '/Users/davidpetrofsk

{
  f: [Function: f],
  g: [Function: g],
  h: [Function: h],
  i: [Function: i],
  j: [Function: j]
}
MyClass {}
<ref *1> Object [global] {
  global: [Circular *1],
  queueMicrotask: [Function: queueMicrotask],
  clearImmediate: [Function: clearImmediate],
  setImmediate: [Function: setImmediate] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  structuredClone: [Getter/Setter],
  clearInterval: [Function: clearInterval],
  clearTimeout: [Function: clearTimeout],
  setInterval: [Function: setInterval],
  setTimeout: [Function: setTimeout] {
    [Symbol(nodejs.util.promisify.custom)]: [Getter]
  },
  atob: [Getter/Setter],
  btoa: [Getter/Setter],
  performance: [Getter/Setter],
  fetch: [AsyncFunction: fetch],
  crypto: [Getter],
  __filename: '[eval]',
  module: <ref *2> [Function: Module] {
    _cache: [Object: null prototype] {},
    _pathCache: [Object: null prototype] {},
    _extensions: [Object: null prototype] {
      '.js': [Function (anonymous)],
      '.json': [

undefined

# Declaration Merging

## Usage in DOM

You can add properties to built-in objects like `window` by defining interfaces for their types (eg. `Window`) to add members, and then setting the new members in your code.

## Interface with Interface

Members of all interfaces of the same name are merged into one.  Duplicates are only considered conflicts if they contradict each other.

In [112]:
(() => {
    interface MyInterface {
        x: number;
        f();
    }
    
    interface MyInterface {
        y: number;
        x: number; // OK because same type, just ignored
        f();  // OK because same thing, ignored
        g();
    }
    
    class MyClass implements MyInterface {
        x: number;
        y: number;
        f() {}
        g() {}
    }
})();

undefined

## Class with Class

__Not allowed__ - if it were allowed that would be __partial classes__.

## Class with Interface

An interface with the same name as a class will add its properties to the class, but they will be undefined until some code sets them on an instance.

In [118]:
(() => {
    class MyClass {
        x: number = 10;
        f() {}
    }
    
    interface MyClass {
        y: number;
        g();
    }
    
    const m = new MyClass();
    m.f();
    console.log(m.y); // undefined, but legal
    console.log(m.g); // undefined, but legal
    m.g = () => {};
    m.g();
})();

undefined
undefined


undefined

## Enum with Enum

Enums can be merged too, but the ordinals will be reundant if you don't explicitly give values.  You could also just give values in one and methods in the other.

In [124]:
(() => {
    enum Color {RED = 0, GREEN, BLUE}
    enum Color {ORANGE = 3, PURPLE}
    
    console.log(Color.RED);
    console.log(Color.PURPLE);
    console.log(Color.GREEN);
})();

0
4
1


undefined

## Extension Method

In [128]:
(() => {
    class MyClass {
        x: number;
        f() {}
    }
    
    // pretend in some other person's code now
    interface MyClass {
        g(); // tell TS to let us add and call this method on MyClass
    }
    
    const m = new MyClass(); // will be affected because prototypes are dynamic!
    
    // not recommended - could also wrap instance creation in a factory
    // then set on instance
    MyClass.prototype.g = () => {console.log('extension!');};
    
    const n = new MyClass();
    m.g();
    n.g();
})();

extension!
extension!


undefined

# Static Members and 'this'

  - Unique to JavaScript, it is illegal to directly call a static method from an instance, or to reference a static variable.
  - When it comes to `this`, JS follows the Python classmethod pattern instead of static method.  The class itself is passed in as `this`.
  - It is illegal to reference an instance member using the class, even if you use `bind` before calling it.

In [18]:
(() => {
    class MyClass {
        static name1 = 'class';
        name2 = 'instance';
        
        static f() {
            return this.name1;
        }

        g() {
            return this.name2;
        }
    }
    
    const m = new MyClass();
    
    // console.log(m.f()); // ILLEGAL
    console.log('m.g(): ' + m.g()); // instance
    console.log('MyClass.f(): ' + MyClass.f()); // class
    // console.log(MyClass.g()); // ILLEGAL
    // console.log(MyClass.g.bind(m)); // ILLEGAL
    console.log('m.g.bind(m): ' + m.g.bind(m)()); // instance
    
    // console.log(m.name1); // ILLEGAL
})();

m.g(): instance
MyClass.f(): class
m.g.bind(m): instance


undefined

# Prototypes Deep-Dive

## Prototype of Instance

All object instances have a __hidden property__ (that you can't directly access) with the prototype of the class for the instance.  You can retrive it with `Object.getPrototypeOf()`.

The default (eg. in this anonymous object) is `Object.prototype`.

In [4]:
(() => {
    const o = {};
    console.log(Object.keys(o));
    console.log(Object.getPrototypeOf(o));
    console.log(Object.prototype);
    console.log(Object.prototype === Object.getPrototypeOf(o));
})();

[]
[Object: null prototype] {}
[Object: null prototype] {}
true


undefined

## Prototype of Class

When you define a class, the class itself (not an instance) has a field called `prototype`, which is not included in iteration, but you can directly access it. The most fundamental default prototype in the system is `Object.prototype`.

Note that prototypes can have both __methods and variables__, but ES6 classes put the variables into the constructor (Python style) instead of the prototype.

In [17]:
(() => {
    class MyClass {
        f() {}
        x: number;
        y = 10;
    }
    
    console.log(MyClass.prototype);
    console.log(Object.prototype);
    
    console.log(Object.keys(MyClass.prototype));
    console.log(Object.keys(MyClass));
})();

{ f: [Function (anonymous)] }
[Object: null prototype] {}
[ 'f' ]
[]


undefined

## Static Members

Static members live on the __class itself__, and __not the prototype__.

That is why statics and instance members are so much more separated in JS than other languages.

In [2]:
(() => {
    class MyClass {
        static f() {}
        g() {}
    }
    
    console.log(MyClass.prototype);
    console.log(Object.keys(MyClass.prototype));
    console.log(Object.keys(MyClass));
})();

{ g: [Function (anonymous)] }
[ 'g' ]
[ 'f' ]


undefined

## Subclasses

The prototype of the base class is located in the secret property of the class's prototype, as if the class itself is an instance of the base class type.  That allows the chain to look uniform.  So you'd call `Object.getPrototypeOf(DerivedClass.prototype)` to get the base class prototype.

Eventually, you reach `Object.prototype`, which has a `null` prototype.

In [10]:
(() => {
    class MyClass {
        f() {}
    }
    
    class DerivedClass extends MyClass {
        f() {}
        g() {}
    }
    
    console.log(DerivedClass.prototype);
    console.log(Object.getPrototypeOf(DerivedClass.prototype));
    console.log(Object.getPrototypeOf(DerivedClass.prototype) === MyClass.prototype);
    
    console.log(Object.getPrototypeOf(Object.getPrototypeOf(DerivedClass.prototype)));
    console.log(Object.getPrototypeOf(Object.getPrototypeOf(Object.getPrototypeOf(DerivedClass.prototype))));
})();

MyClass {
  constructor: [Function: DerivedClass],
  f: [Function (anonymous)],
  g: [Function (anonymous)]
}
{ f: [Function (anonymous)] }
true
[Object: null prototype] {}
null


undefined

## Custom Prototype

You can use any object as a prototype using `Object.create()`  Then it's like the object is the template for the new object. The properties inherited from the object do not show up in enumeration of the object, but they are still accessible via `.` or `[]`.

The reason you can see base class properties when you enumerate a subclass is because `class` does some additional configuration (that you can also do manually) to make the base properties enumerable at the level of the subclass.  This is an ugly detail that may not be worth worrying about until you have to.

In [16]:
(() => {
    const myPrototype = {a: 1, b: 2, f() {}};
    const o = Object.create(myPrototype);
    
    console.log(Object.keys(o)); // a is missing
    console.log(o.a); // but it actually exists
    o.f();
    console.log(o['a']);
})();

[]
1
1


undefined

## Allowed Types of Prototype Object

Non-object types, such as numbers and strings, are not allowed as a prototype, but any other object, including a function, is allowed.

In [22]:
(() => {
    // const o = Object.create(1); // ILLEGAL (even in JS)
    // const o = Object.create('bob'); // ILLEGAL (even in JS)
    const o = Object.create(function() {}); // LEGAL
    const p = Object.create({}); // LEGAL
})();

undefined

## Prototype of a Function

Functions, whether normal functions or lambdas, inherit from `Function`, meaning their prototype is `Function.prototype`.

In [26]:
(() => {
    function f() {}
    const g = () => {};
    
    console.log(Object.getPrototypeOf(f));
    console.log(Object.getPrototypeOf(f) === Function.prototype);
    
    console.log(Object.getPrototypeOf(g));
    console.log(Object.getPrototypeOf(g) === Function.prototype);
})();

{}
true
{}
true


undefined

## Prototype Property on a Function

A function or a function expression, but not an arrow function, automatically gets a `prototype` property on construction, much like a class does.  In the case of functions, this default prototype, which is an independent new object, has a single property called `constructor`, and its own prototype is `Object.prototype`.

The constructor property is just a reference back to the function itself, so that object using the prototype have a way to identify what "class" they are (eg. with `instanceof`).

In [32]:
(() => {
    function f() {}
    const g = () => {};
    const h = function() {};
    
    const o: any = {};
    
    console.log(f.prototype); // exists
    console.log(g.prototype); // shouldn't exist (weird issue in Jupyter)
    console.log(h.prototype); // exists
    console.log(o.prototype); // doesn't exist
    
    console.log(f.prototype === h.prototype); // independent new prototypes
})();

{}
{}
{}
undefined
false


undefined

## Old-Fashioned Classes with 'new'

In the primitive old way of doing it, a function automatically has a `prototype` member pointing back to the function itself.  You add instance members to the class by adding them to the `prototype` member of the function.  You add static members by adding them as properties on the function itself, not the prototype.

Instead of calling the function directly, you invoke it with the `new` keyword, which will do the following:
  1. Construct a new object with the function's prototype as its prototype
  1. Call the function with `this` bound to the new object
  
Thus, effectively, the function whose prototype you modified, is a class (before real classes were added in ES6).

The 2nd cell below shows you could do a similar thing yourself using the low-level machinery of JS.

In [39]:
(() => {
    // function as a class
    function MyClass() {
        this.z = 300;
    }
    
    // instance members
    MyClass.prototype.f = function() {};
    MyClass.prototype.x = 100;
    
    // static members
    MyClass.g = function() {};
    MyClass.y = 200;
    
    // using the "class"
    const o = new MyClass();
    o.f(); // instance
    MyClass.g(); // static
    console.log(o.x);
    console.log(MyClass.y);
    console.log(o.z);
})();

100
200
300


undefined

In [37]:
(() => {
    // Defining our own 'new'.
    function newInstance(ctor: () => void) {
        const o = Object.create(ctor.prototype);
        ctor.call(o);
        
        return o;
    }
    
    // function as a class
    function MyClass() {
        this.z = 300;
    }
    
    // instance members
    MyClass.prototype.f = function() {};
    MyClass.prototype.x = 100;
    
    // static members
    MyClass.g = function() {};
    MyClass.y = 200;
    
    // using the "class"
    const o = newInstance(MyClass);
    o.f(); // instance
    MyClass.g(); // static
    console.log(o.x);
    console.log(MyClass.y);
    console.log(o.z);
})();

100
200
300


undefined

## How Properties are Resolved

If you call `m.x` or `m['x']`, the following steps happen:
1. The property is retrieved on the instance object, m, itself.
1. If it's not there, then it is retrieved from the `prototype` (via the hidden property).
1. If it's not there, JS will iterate backward until it hits `null` in the chain.
1. If it's still not there, then `undefined` will be returned.

Thus, both methods and variables act __virtually__, by virtue of the fact that the latest object in the chain gets the first chance at defining them.  `super()` is just a way to go 1 step back in the chain.

## Inheritance with Old-Fashioned Classes

There is much more manual manipulation of prototypes in inheritance than with a simple class.

The key points to note here are:
1. The base is defined as normal.
1. The derived class function should call the base constructor as a normal function (no `new`) with its own `this` bound.
1. The derived class function's prototype is replaced with a new object inherited from the base's prototype.
1. The `constructor` property has to be manually set because it wasn't automatically created by a function definition.
1. Other than those things, it's the same as defining the base class, and creating with `new`.

In [40]:
(() => {
    function BaseClass() {}
    BaseClass.prototype.x = 100;
    BaseClass.prototype.f = () => {}
    
    function DerivedClass() {
        BaseClass.call(this); // super()
    }
    DerivedClass.prototype = Object.create(BaseClass.prototype);
    DerivedClass.prototype.constructor = DerivedClass;
    DerivedClass.prototype.y = 200;
    DerivedClass.prototype.g = () => {}
    
    const o = new DerivedClass();
    console.log(o.x);
    console.log(o.y);
})();

100
200


undefined

## Diamond Inheritance

Unlike C++ and Python, JavaScript just __does not allow__ that.  There is 1 single __prototype chain__ going from the instance back to `Object`.

TypeScript features like __interfaces__ have nothing to do with this, as they are only for the compiler to help you make less bugs.

## How ES6 Classes Work Underneath

While classes are a first-class language feature, and are not translated into the old-style code, the underlying machinery still works the same way, so it's good to think of it that way.

  - `class MyClass {}` could be thought of as making a function `function MyClass() {}`
  - `extends BaseClass` could be thought of as doing the inheritance hack shown above on that function
  - `constructor(...)` could be thought of as defining the args and body of that function
  - worth noting that the reason classes and functions are the two things that have `prototype` properties (but not instances) is because classes are functions (in essence, underneath)
  - inline definitions of members can be thought of as being set on `MyClass.prototype` after the function definition
    - though sometimes they might be set in the constructor body with `this`
  - static methods go on the class/function instead of the prototype

## Dynamic Nature of Prototypes

Prototypes are not static copies, but are reference objects that are shared across all objects using them.  So if you change it, it will __affect future lookups__ of all the objects, even ones that exist already.

In [46]:
(() => {
    class MyClass {
    }
    
    const o: any = new MyClass();
    console.log(o.x);
    const p: any = new MyClass();
    
    // MyClass.prototype.x = 100; // TS forbirds this but JS would have allowed it
    Object.getPrototypeOf(p).x = 100;
    
    console.log(o.x); // changed p's prototype, and o saw it!
})();

undefined
100


undefined

## Type of Prototype in TS

To allow the old-style to work, prototypes are essentially `any` in TS when you use the old style.  If you use ES6 classes, it will work harder to stop you, but you can still get around it with `Object.getPrototypeOf()`.

## Danger of Modifying Prototypes from Instances

WARNING: always restart the kernel before and after running this cell as it messes with `Object`.

This example shows that by getting and modifying the prototype of a simple empty object, we affected even primitives.  All objects in the system from that point on will have our property because we altered `Object.prototype`.

It is not recommended to modify prototypes that you didn't create like this, especially the built-in ones.

In [52]:
(() => {
    const o = {};
    Object.getPrototypeOf(o).myfakevariable = 1000;
    
    const s: any = 'hi'; // the 'any' is for TypeScript's benefit so it will let us see the JS behavior
    console.log(s.myfakevariable);
})();

1000


undefined

## Anonymous Objects

`const o = {};` will use `Object.prototype` as its prototype.  So it will act as a simple `Object` instance with properties added after the fact.  It will not have its own separate prototype because it does not have a constructor function.  It also cannot be a subclass since it doesn't have its own prototype.

## [[prototype]]

In Chrome DevTools, you may see a member called `[[prototype]]` on an object.  This is the secret hidden field where prototypes are stored on instances, which `Object.getPrototypeOf()` will retrieve.

In general, `[[ ]]` denote internal properties that JS code can't see.

## High-Level Summary

- subclassing and creating an instance of a class are both examples of specializing a template to make it more specific
- JS uses a prototype chain to represent a 1-dimensional chain from `Object` through the inheritance hierarchy to the instance
- when a name is accessed on an object via `.` or `[]`, it is first checked on the object itself, and then one level at a time in the prototype chain until it is found, or `undefined` is returned if never found
- all objects, including classes, functions, anonymous objects, and class instances, have a hidden `[[prototype]]` field (prototype chain parallel to inheritance chain)
  - for class instances, it is the class's prototype
  - for classes, it is the superclass's prototype, down to `Object.prototype`
  - for anonymous objects, it is `Object.prototype`
  - for functions, it is `Function.prototype`
- the prototype of an object can be retrieved with `Object.getPrototypeOf(obj)`
- all functions (inc. lambdas) have a `prototype` property which represents what prototype would be used if that function is used as a class constructor
  - that is separate from the function's own hidden prototype which governs the behavior of the function itself
  - is it auto-populated with a new instance that points back to the function itself as `constructor` property
- old-style classes are made by defining a constructor function and adding members to its prototype, then using the `new` operator to call the constructor function, which automatically applies the prototype first
  - inheritance has to be manually hacked by replacing the derived class prototype with an object prototyped from the base class's prototype
- ES6 classes wrap this process in OOP-friendly syntactic sugar
- `Object.create()` makes an object that uses another object as its prototype
- prototypes are object references and thus dynamic
- static members are on the function/class instead of on the prototype
- there is no diamond inheritance in JS

# Call Signature

A call signature goes in an __interface__ and cannot be implemented by a class.

You can use it, like below, to represent a function, and add properties to that function in a type-protected way.

You can even have multiple call signatures to represent multiple "overloads" (eg. optional parameters, etc.).

In [15]:
(() => {
    interface MyCallable {
        (x: number, y: number): number;
        (x: number): number;
        
        (x: string, y: string): string; // compiler didn't stop us!
        
        f(): void;
    }
    
    function fn(x: number, y: number = 100) {
        return x + y;
    }
    const callable = fn as MyCallable;
    callable.f = () => {};
    
    console.log(callable(10, 20));
    console.log(callable(10));
})();

30
110


undefined

# Construct Signature

A construct signature goes in an __interface__ and cannot be directly implemented by a class - meaning the class cannot directly implement the interface.

But if you cast the class to the interface, the call signature will match propertly if the constructor matches.

Note that the interface knows what the class is, so it's not a base interface really.

This is one way to constrain what you can call a constructor with in JS library code, for instance.

Though you can pass a class directly and call it like a constructor, that way doesn't provide type safety (because you have to pass it as simply `Function`), so that is what this is for.  TypeScript doesn't even allow you to do that (but JS does).

In [25]:
(() => {
    interface MyConstructable {
        new (name: string): MyClass;
        
        name: string;
    }
    
    class MyClass /*implements MyConstructable*/ {
        constructor(readonly name: string) {}
    }
    
    const ctor = MyClass as MyConstructable; // the 'as' isn't actually required here
    const m = new ctor('bob');
    console.log(m.name);
})();

bob


undefined

# Undefined Member

The behavior of a variable that isn't initialized in a class declaration has changed in __ES2020__.  It used to mean nothing - as it does below.

Now, it means overriding with `undefined`.  You can see that in Chrome DevTools by pasting this.

It is not a JS vs. TS difference.

In [2]:
(() => {
    class MyBase {
        x = 5;
        y;
    }
    
    class MyDerived extends MyBase {
        x;
    }
    
    const m = new MyDerived();
    console.log(m.x);
    console.log(m.y);
})();

5
undefined


undefined